CDL Exploration (Nebraska)

Memory-safe CDL checks for Nebraska: per-year corn coverage, downsampled stable-corn mask, and AOI footprint. GPU is used where it helps; disk I/O dominates the per-block counting.

In [ ]:
import sys, pathlib, os
from dotenv import load_dotenv

repo_root = pathlib.Path.cwd().resolve().parent if (pathlib.Path.cwd().name == 'notebooks') else pathlib.Path.cwd().resolve()
sys.path.append(str(repo_root))
load_dotenv(repo_root / '.env')
print('Repo root:', repo_root)
print('GPU_ENABLED:', os.getenv('GPU_ENABLED'))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.enums import Resampling
from shapely.geometry import box
import torch

from src.config import CDL_DIR, GPU_ENABLED
from src.geo.aoi_nebraska import NEBRASKA_BBOX

CORN_CLASS = 1  # CDL corn class id
YEARS = [2019, 2020, 2021, 2022, 2023, 2024]
DOWNSAMPLE = 3  # downsample factor for previews (30m * 3 = 90m pixels)
DEVICE = "cuda" if (GPU_ENABLED and torch.cuda.is_available()) else "cpu"
if GPU_ENABLED and not torch.cuda.is_available():
    raise RuntimeError("GPU_ENABLED=true but CUDA not available; set GPU_ENABLED=false or install CUDA-enabled torch")
print("Using device:", DEVICE)


Per-year corn coverage (streamed)
- Streams CDL blocks to avoid loading multi-GB rasters.
- GPU is used for per-block equality/count if available; still largely disk-bound.
- Reports corn pixels, % of AOI, and km² (30 m pixels → 900 m²).

In [ ]:
def count_corn(path, corn_class=CORN_CLASS):
    """
    Count corn pixels in a CDL raster using block reads.
    Uses GPU for per-block equality/count if DEVICE == "cuda".
    Returns: corn_count (int), total_pixels (int)
    """
    corn_count = 0
    total = 0
    with rasterio.open(path) as src:
        for _, window in src.block_windows(1):
            data = src.read(1, window=window)
            if DEVICE == "cuda":
                t = torch.as_tensor(data, device=DEVICE)
                corn_count += (t == corn_class).sum().item()
                total += t.numel()
            else:
                corn_count += (data == corn_class).sum()
                total += data.size
    return corn_count, total

stats = []
for y in YEARS:
    tif = CDL_DIR / f"cdl_NE_{y}.tif"
    corn_count, total = count_corn(tif)
    pct = corn_count / total * 100
    area_km2 = corn_count * 900 / 1e6  # 30m pixels -> 900 m^2
    stats.append((y, corn_count, pct, area_km2))

print("Per-year corn coverage (corn pixels, % of AOI, km^2 assuming 30m pixels):")
for y, cnt, pct, area in stats:
    print(f"{y}: corn pixels={cnt:,} ({pct:.2f}%), ~{area:.1f} km^2")


Downsampled stable-corn mask
- Resample each year to a coarser grid (factor = DOWNSAMPLE) and compute intersection.
- Shows where corn is stable across all selected years; good sanity check for leakage-free tiling.

In [ ]:
def build_stable_mask_downsample(years, factor=DOWNSAMPLE, corn_class=CORN_CLASS):
    with rasterio.open(CDL_DIR / f"cdl_NE_{years[0]}.tif") as base:
        out_h = base.height // factor
        out_w = base.width // factor
    stable = None
    for y in years:
        with rasterio.open(CDL_DIR / f"cdl_NE_{y}.tif") as src:
            data = src.read(
                1,
                out_shape=(out_h, out_w),
                resampling=Resampling.nearest,
            )
            mask = data == corn_class
            stable = mask if stable is None else (stable & mask)
    return stable

stable_mask_ds = build_stable_mask_downsample(YEARS)
stable_count = stable_mask_ds.sum()
stable_pct = stable_count / stable_mask_ds.size * 100
stable_area_km2 = stable_count * 900 * (DOWNSAMPLE ** 2) / 1e6
print(f"Stable corn (downsampled {DOWNSAMPLE}x): {stable_count:,} pixels ({stable_pct:.2f}%), ~{stable_area_km2:.1f} km^2")

plt.figure(figsize=(8, 8))
plt.imshow(stable_mask_ds, cmap="gray")
plt.title(f"Stable Corn Mask (factor {DOWNSAMPLE}, ~{30*DOWNSAMPLE}m pixels)")
plt.axis("off")
plt.show()


Corn masks by year (downsampled previews)

Downsampled visuals (factor = DOWNSAMPLE) to spot spatial inconsistencies without loading full resolution.

In [ ]:
with rasterio.open(CDL_DIR / f"cdl_NE_{YEARS[0]}.tif") as src0:
    out_h = src0.height // DOWNSAMPLE
    out_w = src0.width // DOWNSAMPLE

fig, axes = plt.subplots(1, len(YEARS), figsize=(3 * len(YEARS), 4))
for i, y in enumerate(YEARS):
    with rasterio.open(CDL_DIR / f"cdl_NE_{y}.tif") as src:
        data = src.read(1, out_shape=(out_h, out_w), resampling=Resampling.nearest)
        corn_mask = data == CORN_CLASS
        ax = axes[i]
        ax.imshow(corn_mask, cmap="gray")
        ax.set_title(f"Corn {y}")
        ax.axis("off")
plt.suptitle(f"CDL Corn Masks (downsample {DOWNSAMPLE}x, ~{30*DOWNSAMPLE}m)", y=1.02)
plt.tight_layout()
plt.show()


AOI footprint

Bounding box for the Nebraska study area (WGS84).

In [ ]:
poly = box(*NEBRASKA_BBOX)
x, y = poly.exterior.xy
plt.figure(figsize=(5, 5))
plt.plot(x, y)
plt.title("Nebraska AOI Bounding Box (WGS84)")
plt.axis("equal")
plt.grid(True)
plt.show()


Notes
- Run cells in order; the imports/constants cell defines CORN_CLASS, DEVICE, etc.
- GPU helps for model training; this notebook is I/O-bound, so GPU acceleration on counts is minimal.
- If imports fail, ensure kernel is `.venv/bin/python` and run `%pip install -r requirements.txt`.